In [ ]:
! pip install geopy

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

In [ ]:
df=pd.read_csv('data/CoffeeBean.csv')
df.address
geolocoder = Nominatim(user_agent='South Korea')

In [ ]:
a= ['서울특별시 강남구 학동로 211', '서울특별시 강남구 광평로 280', '서울특별시 강남구 논현로 556']
for i, loc in enumerate(a):
    geo=geolocoder.geocode(loc)
    crd=(geo.latitude, geo.longitude)
    print(geo, crd)
    

In [ ]:
data=pd.read_csv('data/보건복지부_공공보건 의료기관 현황_20161231.csv')
data

In [ ]:
addr=pd.DataFrame(data['주소'].apply(lambda v: v.split()[:2]).to_list(),columns=('시도','군구'))
addr

In [ ]:
addr['시도'].unique()

In [ ]:
addr[addr['시도']=='창원시']

In [ ]:
addr.iloc[27]=['경상남도', '창원시']
addr.iloc[31]=['경상남도', '창원시']
addr['시도'].unique()

In [ ]:
addr[addr['시도']=='경산시']
addr.iloc[47]=['경상북도','경산시']
addr['시도'].unique()

In [ ]:
addr[addr['시도']=='천안시']
addr.iloc[209]=['충청남도','천안시']
addr.iloc[210]=['충청남도','천안시']
addr['시도'].unique()

In [ ]:
addr_aliases={'경기':'경기도','경남':'경상남도','경북':'경상북도','충북':'충청북도','충남':'충청남도','전남':'전라남도','전북':'전라북도','서울시':'서울특별시','부산특별시':'부산광역시',
              '대전': '대전광역시'}

In [ ]:
addr['시도']=addr['시도'].apply(lambda v: addr_aliases.get(v, v))
addr['시도'].unique()

In [ ]:
addr[addr['군구']=='아란13길']

In [ ]:
addr.iloc[75]=['제주특별자치구','제주시']

In [ ]:
addr[addr['군구']=='동매로']
addr.iloc[155]=['부산광역시','사하구']

In [ ]:
addr['군구'].unique()

In [ ]:
addr['시도군구']=addr.apply(lambda r: r['시도']+' '+r['군구'], axis=1)
addr['count']=0
addr.loc[addr['시도군구']=='강원도 강릉시']

In [ ]:
addr_group=pd.DataFrame(addr.groupby(['시도','군구','시도군구'], as_index=False).count())
addr_group=addr_group.set_index('시도군구')
addr_group

In [ ]:
population=pd.read_csv('data/행정구역_시군구_별__성별_인구수_20230717102216.csv')
population.head()

In [ ]:
population=population.rename(columns={'행정구역시도':'시도','행정구역군구':'군구',
                                      '남자인구수':'남자','여자인구수':'여자'})
population.head()

In [ ]:
population['시도'].unique()
population['군구'].unique()

In [ ]:
population['시도군구'] = population.apply(lambda r : r['시도']+' '+r['군구'],axis=1)
popluation = population[population['군구'] != '소계'] #popluation = popluation[popluation.'군구' != '소계']
popluation = popluation.set_index('시도군구')
popluation = popluation[popluation['시도'] != '전국']

In [ ]:
addr_popluation = pd.merge(addr_group,popluation,how='inner', left_index=True, right_index=True) #inner 조인
addr_popluation

In [ ]:
geolocoder = Nominatim(user_agent='South Korea')

In [ ]:
addr_poplulation2=addr_popluation[['시도_x','군구_x','count','총인구수']]
addr_poplulation2=addr_poplulation2.rename(columns={'시도_x':'시도','군구_x':'군구','총인구수':'인구수'})
addr_poplulation2

In [ ]:
counts=addr_poplulation2['count']
addr_poplulation2['ratio']=counts.divide(addr_poplulation2['인구수'], axis=0)*100000
addr_poplulation2

In [ ]:
data_draw_korea=pd.read_csv('data/data_draw_korea.csv',index_col=0)
data_draw_korea

In [ ]:
data_draw_korea['시도군구']=data_draw_korea.apply(lambda r:r['광역시도']+ ' '+ r['행정구역'], axis=1)
data_draw_korea=data_draw_korea.set_index('시도군구')
data_draw_korea

In [ ]:
data_population=pd.merge(data_draw_korea, addr_poplulation2, how='outer',left_index=True,right_index=True)
data_population

In [ ]:
BORDER_LINES = [
    [(3, 2), (5, 2), (5, 3), (9, 3), (9, 1)], # 인천
    [(2, 5), (3, 5), (3, 4), (8, 4), (8, 7), (7, 7), (7, 9), (4, 9), (4, 7), (1, 7)], # 서울
    [(1, 6), (1, 9), (3, 9), (3, 10), (8, 10), (8, 9),
     (9, 9), (9, 8), (10, 8), (10, 5), (9, 5), (9, 3)], # 경기도
    [(9, 12), (9, 10), (8, 10)], # 강원도
    [(10, 5), (11, 5), (11, 4), (12, 4), (12, 5), (13, 5),
     (13, 4), (14, 4), (14, 2)], # 충청남도
    [(11, 5), (12, 5), (12, 6), (15, 6), (15, 7), (13, 7),
     (13, 8), (11, 8), (11, 9), (10, 9), (10, 8)], # 충청북도
    [(14, 4), (15, 4), (15, 6)], # 대전시
    [(14, 7), (14, 9), (13, 9), (13, 11), (13, 13)], # 경상북도
    [(14, 8), (16, 8), (16, 10), (15, 10),
     (15, 11), (14, 11), (14, 12), (13, 12)], # 대구시
    [(15, 11), (16, 11), (16, 13)], # 울산시
    [(17, 1), (17, 3), (18, 3), (18, 6), (15, 6)], # 전라북도
    [(19, 2), (19, 4), (21, 4), (21, 3), (22, 3), (22, 2), (19, 2)], # 광주시
    [(18, 5), (20, 5), (20, 6)], # 전라남도
    [(16, 9), (18, 9), (18, 8), (19, 8), (19, 9), (20, 9), (20, 10)], # 부산시
]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import rc, font_manager
fontname='c:/Windows/Fonts/malgun.ttf'
font_name=font_manager.FontProperties(fname=fontname).get_name()
rc('font',family=font_name)

In [ ]:
def draw_blockMap(blockedMap, targetData, title, color ):
    whitelabelmin = (max(blockedMap[targetData]) - min(blockedMap[targetData])) * 0.25 + min(blockedMap[targetData])

    datalabel = targetData

    vmin = min(blockedMap[targetData])
    vmax = max(blockedMap[targetData])

    mapdata = blockedMap.pivot(index='y', columns='x', values=targetData)
    masked_mapdata = np.ma.masked_where(np.isnan(mapdata), mapdata)
    
    plt.figure(figsize=(8, 13))
    plt.title(title)
    plt.pcolor(masked_mapdata, vmin=vmin, vmax=vmax, cmap=color, edgecolor='#aaaaaa', linewidth=0.5)

# 지역 이름 표시
    for idx, row in blockedMap.iterrows():
        annocolor = 'white' if row[targetData] > whitelabelmin else 'black'
    
        # 광역시는 구 이름이 겹치는 경우가 많아서 시단위 이름도 같이 표시한다. (중구, 서구)
        if row['광역시도'].endswith('시') and not row['광역시도'].startswith('세종'):
            dispname = '{}\n{}'.format(row['광역시도'][:2], row['행정구역'][:-1])
            if len(row['행정구역']) <= 2:
                dispname += row['행정구역'][-1]
        else:
            dispname = row['행정구역'][:-1]

        # 서대문구, 서귀포시 같이 이름이 3자 이상인 경우에 작은 글자로 표시한다.
        if len(dispname.splitlines()[-1]) >= 3:
            fontsize, linespacing = 9.5, 1.5
        else:
            fontsize, linespacing = 11, 1.2

        plt.annotate(dispname, (row['x']+0.5, row['y']+0.5), weight='bold',
                     fontsize=fontsize, ha='center', va='center', color=annocolor,
                     linespacing=linespacing)
    
    # 시도 경계 그린다.
    for path in BORDER_LINES:
        ys, xs = zip(*path)
        plt.plot(xs, ys, c='black', lw=4)

    plt.gca().invert_yaxis()
    #plt.gca().set_aspect(1)
    plt.axis('off')
    
    cb = plt.colorbar(shrink=.1, aspect=10)
    cb.set_label(datalabel)

    plt.tight_layout()
    
    plt.savefig('data/blockMap_' + targetData + '.png')
                    
    plt.show()

In [ ]:
draw_blockMap(data_population,'count','행정구역별 공공보건의료기관 수','Blues')

In [ ]:
draw_blockMap(data_population, 'ratio', '행정구역별 인구수대비 공공보건의료기간수','Reds')